In [15]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pickle

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.transforms as transforms

#from scipy.signal import lfilter
#from math import atan2, degrees, radians

import bee_simulator
import central_complex
import cx_basic
import cx_rate

import analysis
import plotter2

import modules

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Environment
nest_radius = 15   # Area in which the agent feels like home
feeder_radius = 20

In [31]:
# Set the cx model type and logger type here for easy access

cx = cx_rate.CXRatePontinHolonomic(noise=0.1)   # Latest CX model with pontine cells + holonomic movement
CXLogger = modules.trials2.CXLogger

# Save / Load data

In [ ]:
def generate_fw_filename(Distances, Run_Success, **kwargs):
    
    filename = '{0}-to-{1}-by{2}_N{3}'.format(str(Distances[0]),
                                              str(Distances[-1]),
                                              str(len(Distances)),
                                              str(Run_Success.shape[0]))
    for k, v in kwargs.iteritems():
        filename += '_' + k + str(v)
    return filename + '.npz'

def save_fw_data(Distances, Run_Success, Raw_Angular_Distances, Clean_Angular_Distances, Straightness_List, Real_Distances, Estim_Distances, Amplitudes, **kwargs):
    filename = generate_fw_filename(Distances, Run_Success, **kwargs)
    
    np.savez(os.path.join(DATA_PATH, filename),
             Distances = Distances,
             Run_Success = Run_Success,
             Raw_Angular_Distances = Raw_Angular_Distances,
             Clean_Angular_Distances = Clean_Angular_Distances,
             Straightness_List = Straightness_List,
             Real_Distances = Real_Distances,
             Estim_Distances = Estim_Distances, 
             Amplitudes = Amplitudes
            )
    
def load_fw_data(filename=None):
    
    if filename is None:
        return None
    else:
        
        with np.load(os.path.join(DATA_PATH, filename)) as data:
            Distances = data['Distances'],
            Run_Success = data['Run_Success'],
            Raw_Angular_Distances = data['Raw_Angular_Distances'],
            Clean_Angular_Distances = data['Clean_Angular_Distances'],
            Straightness_List = data['Straightness_List']

        return Distances[0], Run_Success[0], Raw_Angular_Distances[0], Clean_Angular_Distances[0], Straightness_List

# Repeat food-ward experiment

In [ ]:
# Do a lot of trials and return success, angular distance and other things

N = 1000
N_dists = 30

max_dist = 20000
min_dist = 500
gain = 1.05

Distances = np.round(np.linspace(min_dist, max_dist, num=N_dists)).astype(int)

Run_Success = np.zeros((N, N_dists))
Raw_Angular_Distances = np.zeros((N, N_dists))
Clean_Angular_Distances = np.zeros((N, N_dists))
Straightness_List = np.empty((N_dists, 500, N))

Real_Distances = np.zeros((N, N_dists))
Estim_Distances = np.zeros((N, N_dists))
Amplitudes = np.zeros((N, N_dists))

for d, distance in enumerate(Distances):
    print distance
    
    run_success = np.zeros(N)
    raw_ang_dist = np.zeros(N)
    clean_ang_dist = np.zeros(N)
    straightness_list = np.empty((500, N))
    
    real_distances = np.zeros(N)
    estim_distances = np.zeros(N)
    amplitudes = np.zeros(N)

    for i in range(N):
        
        if i % 100 == 0:
            print (i)
        
        h_p1, v_p1, T_p1, log_p1, success = move(goal_coord=None,
                                                T_max=distance,
                                                goal_radius=feeder_radius,
                                                random_exploring=True,
                                                arena=False,
                                                stop_when_food=True,
                                                logging=True,
                                                return_success=True
                                                )

        goal = np.cumsum(v_p1, axis=0)[-1]

        h_p2, v_p2, T_p2, log_p2, detection = move(goal_coord=goal,
                                                T_max=distance,
                                                ltm=log_p1.memory[:,-1],
                                                goal_radius=feeder_radius,
                                                random_exploring=False,
                                                arena=False,
                                                stop_when_food=True,
                                                logging=True,
                                                return_success=False,
                                                inhib_gain=gain,
                                                keep_searching=True
                                                )

        if len(detection) > 0:
            success = 1
        else:
            success = 0

        raw_angle = analysis2.simple_angular_distance(v_p2, goal, nb_steps=20)
        clean_angle = analysis2.simple_angular_distance(v_p2, goal, nb_steps=20, nest_radius=nest_radius)
        straightness, real_D = analysis2.compute_simple_path_straightness(v_p2, goal, return_dist=True)
        
        #estim_D = get_estimated_distance(cpu4=log_p2.cpu4[:,0], ltm=log_p1.memory[:,-1], cx=cx)
        #amplitude = get_amplitude(cx.cpu4_inhibition(log_p2.cpu4[:,0], log_p1.memory[:,-1], gain=gain))
        
        run_success[i] = success
        raw_ang_dist[i] = raw_angle
        clean_ang_dist[i] = clean_angle
        straightness_list[:,i] = straightness[0]

        real_distances[i] = real_D
        #estim_distances[i] = estim_D         
        #amplitudes[i] = amplitude

        
    Run_Success[:, d] = run_success
    Raw_Angular_Distances[:, d] = raw_ang_dist
    Clean_Angular_Distances[:, d] = clean_ang_dist
    Straightness_List[d, :, :] = straightness_list
    
    Real_Distances[:, d] = real_distances
    #Estim_Distances[:, d] = estim_distances
    #Amplitudes[:, d] = amplitudes

np.nanmean(Run_Success, axis=0)*100, Distances

In [ ]:
# Save current results
save_fw_data(Distances, Run_Success, Raw_Angular_Distances, Clean_Angular_Distances, Straightness_List, Real_Distances, Estim_Distances, Amplitudes)

In [ ]:
# Load previous results (format .npz)

# Exemple, loading two sets of data
Distances_1, Run_Success_1, Raw_Angular_Distances_1, Clean_Angular_Distances_1, Straightness_List_1 = load_fw_data(filename='100-to-2200-by10_N1000_gain1.05.npz')
Distances_2, Run_Success_2, Raw_Angular_Distances_2, Clean_Angular_Distances_2, Straightness_List_2 = load_fw_data(filename='100-to-5000-by10_N1000_gain1.05.npz')

In [ ]:
# Concatenate and sort the two sets of distances + success to make one big set
d = np.concatenate((Distances_1, Distances_2[1:]), axis=0) # We ignore the first one in the second set here because it's already in the first set
D = np.sort(d)

s = np.concatenate((Run_Success_1, Run_Success_2[:,1:]), axis=1)
S = s[:,d.argsort()]

In [ ]:
plotter2.plot_success(D, S, color='purple')

In [ ]:
colors = [plotter2.cm.viridis(x) for x in np.linspace(0, 1, 19)]

fig, ax = plotter2.plot_angular_distance_histogram(RA[:,0], figsize=(8,8), bins=72, color=colors[0])

for i in range(1,19):
    plotter2.plot_angular_distance_histogram(RA[:,i], bins=72, color=colors[i], ax=ax)

cbax = fig.add_axes([1.02, 0.05, 0.02, 0.9])
m = cm.ScalarMappable(cmap='viridis')
m.set_array(np.linspace(100, 5000))
fig.colorbar(m, cbax, ticks=[100, 1000, 2000, 3000, 4000, 5000])

# Generate a lot of random walks (for shortcutting tests)

In [ ]:
R = 200
n = 10

D = np.round(np.linspace(5000, 20000, num=n)).astype(int)

goals = np.zeros((R, 2))
mems = np.zeros((R, 16))

Mems = np.zeros((R, len(D), 16))
Goals = np.zeros((R, len(D), 2))

for d, distance in enumerate(D):
    for i in range(R):

        h_p1, v_p1, T_p1, log_p1, success = modules.move(goal_coord=None,
                                                         T_max=distance,
                                                         goal_radius=feeder_radius,
                                                         random_exploring=True,
                                                         arena=False,
                                                         stop_when_food=True,
                                                         logging=True,
                                                         return_success=True
                                                        )
        mems[i,:] = log_p1.memory[:,-1]
        goals[i,:] = np.cumsum(v_p1, axis=0)[-1]
    Mems[:, d, :] = mems
    Goals[:, d, :] = goals

Mlist = Mems.reshape(n*R,16)
Glist = Goals.reshape(n*R,2)

In [ ]:
# Save current feeder cloud data (memories and coordinates)

with open('feeder_pool_coords.pickle', 'wb') as handle:
    pickle.dump(Glist, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('feeder_pool_mems.pickle', 'wb') as handle:
    pickle.dump(Mlist, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load previous feeder cloud data (memories and coordinates)

with open('feeder_pool_coords.pickle', 'rb') as handle:
    Glist = pickle.load(handle)
    
with open('feeder_pool_mems.pickle', 'rb') as handle:
    Mlist = pickle.load(handle)

In [ ]:
# Get all distances between all couples of feeders

from math import hypot
from itertools import combinations

list_of_all_coords = []

for f in range(0, 99):
    x, y = Glist[f,:]
    
    list_of_all_coords.append((x,y))

def eucldistance(p1, p2):
    """Euclidean distance between two points."""
    x1, y1 = p1
    x2, y2 = p2
    return hypot(x2 - x1, y2 - y1)

list_of_all_dists = [eucldistance(*combo) for combo in combinations(list_of_all_coords, 2)]

In [ ]:
plotter2.plot_feeders_cloud(Glist, color='r', figsize=(8,8))
plotter2.plot_feeders_hist(Glist, color='g', figsize=(8,8))
plotter2.plot_relative_feeders_hist(list_of_all_dists, color='purple', figsize=(8,8))

In [ ]:
# Generate 1000 shortcut trials.
# For that, pick a random couple of feeders from the cloud and try a food-ward route.
# If food-ward route is successful, try the shortcut. If not, retry a new food-ward route, etc.

N = 1001

nb = G.shape[0]

i = 0
run_success = np.zeros(N)
first_t = np.zeros(N)
raw_ang_dist = np.zeros(N)
clean_ang_dist = np.zeros(N)
straightness_list = np.empty((500, N))

real_distances = np.zeros(N)
estim_distances = np.zeros(N)
amplitudes = np.zeros(N)

while i < N-1:

    detection_p1 = 0
    detection_p2 = 0
    success = 0
    
    # This bit is to ensure the distance between the two picked feeders is not too small or too big
    while True:
        couple = np.random.choice(nb, 2, replace=False)
        feederA = G[couple[0]]
        feederB = G[couple[1]]
        
        if 10000 > fdistance(feederA, feederB) > 100:
            break

    memA = M[couple[0]]
    memB = M[couple[1]]

    h_p1, v_p1, T_p1, log_p1, detection_p1 = modules.move(goal_coord=feederA,
                                                          T_max=5000,
                                                          ltm=memA,
                                                          goal_radius=20,
                                                          random_exploring=False,
                                                          arena=False,
                                                          stop_when_food=True,
                                                          logging=True,
                                                          return_success=True,
                                                          inhib_gain=1.05,
                                                          keep_searching=False
                                                         )
    
    if detection_p1 == 1:
        i += 1
        
        h_p2, v_p2, T_p2, log_p2, detection_p2 = modules.move(goal_coord=feederB,
                                                              start_coord=feederA,
                                                              T_max=5000,
                                                              memory=log_p1.memory[:,-1],
                                                              tb1=log_p1.tb1[:,-1],
                                                              ltm=memB,
                                                              goal_radius=20,
                                                              random_exploring=False,
                                                              arena=False,
                                                              stop_when_food=True,
                                                              logging=True,
                                                              return_success=True,
                                                              inhib_gain=1.05,
                                                              keep_searching=False
                                                             )
        if detection_p2 == 1:
            success = 1

        else:
            success = 0
    else:
        success = 0

    raw_angle = analysis2.simple_angular_distance(v_p2, feederB, start=feederA, nb_steps=20)
    clean_angle = analysis2.simple_angular_distance(v_p2, feederB, start=feederA, nb_steps=20, nest_radius=nest_radius)
    straightness, real_D = analysis2.compute_simple_path_straightness(v_p2, feederB, start=feederA, return_dist=True)

    #estim_D = get_estimated_distance(cpu4=log_p2.cpu4[:,0], ltm=log_p1.memory[:,-1], cx=cx)
    #amplitude = get_amplitude(log_p2.cpu4_inh[:,0])

    run_success[i] = success

    raw_ang_dist[i] = raw_angle
    clean_ang_dist[i] = clean_angle
    straightness_list[:,i] = straightness[0]

    real_distances[i] = real_D
    #estim_distances[i] = estim_D         
    #amplitudes[i] = amplitude

In [ ]:
fig, ax = plotter2.plot_fw_route_straightness(straightness, color='purple')